In [2]:
import os
import cv2
from tqdm import tqdm

# Ruta base donde se encuentran los datos
base_path = r'C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas'

def find_generated_videos():
    """
    Busca todos los videos MP4 en las carpetas de muestras
    
    Returns:
        list: Lista de tuplas (expresión, muestra, nombre_video, ruta_completa)
    """
    videos_found = []
    
    # Recorrer todas las carpetas de expresiones
    for expression in os.listdir(base_path):
        expression_path = os.path.join(base_path, expression)
        
        if not os.path.isdir(expression_path):
            continue
            
        # Recorrer las carpetas de muestras
        try:
            for muestra_folder in os.listdir(expression_path):
                muestra_path = os.path.join(expression_path, muestra_folder)
                
                if not os.path.isdir(muestra_path) or not muestra_folder.startswith('muestra_'):
                    continue
                    
                # Buscar archivos MP4 en la carpeta de muestra
                for file in os.listdir(muestra_path):
                    if file.endswith('.mp4'):
                        video_path = os.path.join(muestra_path, file)
                        videos_found.append((expression, muestra_folder, file, video_path))
        except Exception as e:
            print(f"Error al explorar {expression_path}: {e}")
    
    return videos_found

def display_videos(videos):
    """
    Muestra la lista de videos encontrados con detalles
    
    Args:
        videos: Lista de tuplas (expresión, muestra, nombre_video, ruta_completa)
    """
    if not videos:
        print("❌ No se encontraron videos generados.")
        return
    
    print(f"📊 Videos encontrados ({len(videos)}):\n")
    
    for i, (expression, muestra, video, video_path) in enumerate(videos, 1):
        video_size_mb = os.path.getsize(video_path) / (1024 * 1024)
        
        # Obtener información del video usando OpenCV
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"{i}. {expression}/{muestra}/{video} - ⚠️ No se pudo abrir el video")
            continue
            
        # Obtener propiedades
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration = frame_count / fps if fps > 0 else 0
        
        cap.release()
        
        print(f"{i}. {expression}/{muestra}/{video}")
        print(f"   - Tamaño: {video_size_mb:.2f} MB")
        print(f"   - Resolución: {width}x{height}")
        print(f"   - FPS: {fps:.2f}")
        print(f"   - Frames: {frame_count}")
        print(f"   - Duración: {duration:.2f} segundos\n")

def delete_all_videos(videos):
    """
    Elimina todos los videos de la lista
    
    Args:
        videos: Lista de tuplas (expresión, muestra, nombre_video, ruta_completa)
    
    Returns:
        int: Número de videos eliminados
    """
    if not videos:
        print("❌ No hay videos para eliminar.")
        return 0
    
    deleted_count = 0
    
    for _, _, video_name, video_path in tqdm(videos, desc="Eliminando videos"):
        try:
            os.remove(video_path)
            deleted_count += 1
        except Exception as e:
            print(f"⚠️ No se pudo eliminar {video_path}: {e}")
    
    return deleted_count

def delete_videos_by_expression(videos, target_expression):
    """
    Elimina los videos de una expresión específica
    
    Args:
        videos: Lista de tuplas (expresión, muestra, nombre_video, ruta_completa)
        target_expression: Nombre de la expresión cuyos videos se quieren eliminar
    
    Returns:
        int: Número de videos eliminados
    """
    if not videos:
        print("❌ No hay videos para eliminar.")
        return 0
    
    # Filtrar videos por expresión
    videos_to_delete = [v for v in videos if v[0].lower() == target_expression.lower()]
    
    if not videos_to_delete:
        print(f"❌ No se encontraron videos para la expresión '{target_expression}'.")
        return 0
    
    # Mostrar videos que se van a eliminar
    print(f"Se eliminarán {len(videos_to_delete)} videos de la expresión '{target_expression}':")
    for i, (_, muestra, video, _) in enumerate(videos_to_delete, 1):
        print(f"  {i}. {muestra}/{video}")
    
    # Confirmar eliminación
    confirm = input("\n¿Estás seguro de que deseas eliminar estos videos? (s/n): ")
    if confirm.lower() != 's':
        print("❌ Operación cancelada.")
        return 0
    
    deleted_count = 0
    
    for _, _, video_name, video_path in tqdm(videos_to_delete, desc=f"Eliminando videos de '{target_expression}'"):
        try:
            os.remove(video_path)
            deleted_count += 1
        except Exception as e:
            print(f"⚠️ No se pudo eliminar {video_path}: {e}")
    
    return deleted_count

def delete_specific_videos(videos):
    """
    Permite al usuario seleccionar videos específicos para eliminar
    
    Args:
        videos: Lista de tuplas (expresión, muestra, nombre_video, ruta_completa)
    
    Returns:
        int: Número de videos eliminados
    """
    if not videos:
        print("❌ No hay videos para eliminar.")
        return 0
    
    # Mostrar videos numerados
    print("Selecciona los videos que deseas eliminar (ingresa los números separados por comas):\n")
    for i, (expression, muestra, video, _) in enumerate(videos, 1):
        print(f"{i}. {expression}/{muestra}/{video}")
    
    # Solicitar selección
    selection = input("\nNúmeros de videos a eliminar (ej: 1,3,5-7): ")
    
    # Procesar selección
    selected_indices = set()
    
    for part in selection.split(','):
        if '-' in part:
            # Rango (ej: 5-7)
            try:
                start, end = map(int, part.split('-'))
                if 1 <= start <= len(videos) and 1 <= end <= len(videos):
                    selected_indices.update(range(start, end + 1))
            except ValueError:
                print(f"⚠️ Rango inválido: {part}")
        else:
            # Número individual
            try:
                index = int(part)
                if 1 <= index <= len(videos):
                    selected_indices.add(index)
            except ValueError:
                print(f"⚠️ Número inválido: {part}")
    
    if not selected_indices:
        print("❌ No se seleccionaron videos válidos para eliminar.")
        return 0
    
    # Convertir a lista y ordenar
    selected_indices = sorted(list(selected_indices))
    
    # Confirmar eliminación
    print(f"\nSe eliminarán {len(selected_indices)} videos:")
    for idx in selected_indices:
        expression, muestra, video, _ = videos[idx - 1]
        print(f"  - {expression}/{muestra}/{video}")
    
    confirm = input("\n¿Estás seguro de que deseas eliminar estos videos? (s/n): ")
    if confirm.lower() != 's':
        print("❌ Operación cancelada.")
        return 0
    
    # Eliminar videos seleccionados
    deleted_count = 0
    
    for idx in selected_indices:
        _, _, video_name, video_path = videos[idx - 1]
        try:
            os.remove(video_path)
            deleted_count += 1
            print(f"✅ Eliminado: {video_path}")
        except Exception as e:
            print(f"⚠️ No se pudo eliminar {video_path}: {e}")
    
    return deleted_count

# Menú principal
if __name__ == "__main__":
    print("🔍 Buscando videos generados...")
    videos = find_generated_videos()
    
    if not videos:
        print("❌ No se encontraron videos generados en la estructura de carpetas.")
        exit()
    
    while True:
        print("\n===== ELIMINADOR DE VIDEOS GENERADOS =====")
        print(f"Videos encontrados: {len(videos)}")
        print("\n¿Qué acción deseas realizar?")
        print("1. Ver la lista de videos")
        print("2. Eliminar todos los videos")
        print("3. Eliminar videos de una expresión específica")
        print("4. Eliminar videos específicos")
        print("5. Salir")
        
        choice = input("\nIngresa tu opción (1-5): ")
        
        if choice == "1":
            display_videos(videos)
        
        elif choice == "2":
            confirm = input(f"¿Estás seguro de que deseas eliminar TODOS ({len(videos)}) los videos? (s/n): ")
            if confirm.lower() == 's':
                deleted = delete_all_videos(videos)
                print(f"\n✅ {deleted} videos fueron eliminados.")
                if deleted > 0:
                    # Actualizar la lista de videos
                    videos = find_generated_videos()
            else:
                print("❌ Operación cancelada.")
        
        elif choice == "3":
            # Mostrar expresiones disponibles
            expressions = sorted(set(v[0] for v in videos))
            print("\nExpresiones disponibles:")
            for i, exp in enumerate(expressions, 1):
                count = sum(1 for v in videos if v[0] == exp)
                print(f"{i}. {exp} ({count} videos)")
            
            exp_choice = input("\nIngresa el nombre de la expresión: ")
            
            deleted = delete_videos_by_expression(videos, exp_choice)
            if deleted > 0:
                print(f"\n✅ {deleted} videos fueron eliminados.")
                # Actualizar la lista de videos
                videos = find_generated_videos()
        
        elif choice == "4":
            deleted = delete_specific_videos(videos)
            if deleted > 0:
                print(f"\n✅ {deleted} videos fueron eliminados.")
                # Actualizar la lista de videos
                videos = find_generated_videos()
        
        elif choice == "5":
            print("👋 ¡Hasta luego!")
            break
        
        else:
            print("❌ Opción inválida. Por favor, intenta de nuevo.")

🔍 Buscando videos generados...

===== ELIMINADOR DE VIDEOS GENERADOS =====
Videos encontrados: 166

¿Qué acción deseas realizar?
1. Ver la lista de videos
2. Eliminar todos los videos
3. Eliminar videos de una expresión específica
4. Eliminar videos específicos
5. Salir
📊 Videos encontrados (166):

1. A ver/muestra_1/a_ver_muestra_1.mp4
   - Tamaño: 0.70 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 69
   - Duración: 2.30 segundos

2. A ver/muestra_1/video_a_ver_muestra_1.mp4
   - Tamaño: 0.40 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 69
   - Duración: 2.30 segundos

3. A ver/muestra_2/a_ver_muestra_2.mp4
   - Tamaño: 0.72 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 70
   - Duración: 2.33 segundos

4. A ver/muestra_2/video_a_ver_muestra_2.mp4
   - Tamaño: 0.42 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 70
   - Duración: 2.33 segundos

5. A ver/muestra_3/a_ver_muestra_3.mp4
   - Tamaño: 0.62 MB
   - Resolución: 640x640
   - FPS: 30.00

In [1]:
import os
import glob
import shutil
import time

def delete_keyframe_folders(base_path):
    """
    Elimina todas las carpetas llamadas "Keyframe" en la estructura de directorios
    
    Args:
        base_path: Ruta base donde buscar las carpetas Keyframe
    
    Returns:
        Dict: Estadísticas de las carpetas eliminadas
    """
    start_time = time.time()
    
    # Estadísticas
    stats = {
        'total_folders_found': 0,
        'folders_deleted': 0,
        'folders_failed': 0,
        'details': []
    }
    
    # Buscar todas las carpetas "Keyframe"
    keyframe_pattern = os.path.join(base_path, "**", "Keyframe")
    keyframe_folders = glob.glob(keyframe_pattern, recursive=True)
    
    stats['total_folders_found'] = len(keyframe_folders)
    
    if not keyframe_folders:
        print(f"❌ No se encontraron carpetas 'Keyframe' en: {base_path}")
        return stats
    
    print(f"🔍 Se encontraron {len(keyframe_folders)} carpetas 'Keyframe'.")
    
    # Pedir confirmación
    confirm = input(f"¿Estás seguro de que deseas eliminar {len(keyframe_folders)} carpetas 'Keyframe'? (s/n): ")
    if confirm.lower() != 's':
        print("❌ Operación cancelada.")
        return stats
    
    # Eliminar cada carpeta
    for i, folder_path in enumerate(keyframe_folders, 1):
        try:
            print(f"[{i}/{len(keyframe_folders)}] Eliminando: {folder_path}")
            
            # Contar archivos
            file_count = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
            
            # Eliminar la carpeta y su contenido
            shutil.rmtree(folder_path)
            
            stats['folders_deleted'] += 1
            stats['details'].append({
                'path': folder_path,
                'files': file_count,
                'success': True,
                'error': None
            })
            
            print(f"✅ Carpeta eliminada ({file_count} archivos)")
            
        except Exception as e:
            print(f"❌ Error al eliminar {folder_path}: {str(e)}")
            stats['folders_failed'] += 1
            stats['details'].append({
                'path': folder_path,
                'success': False,
                'error': str(e)
            })
    
    # Mostrar resumen
    total_time = time.time() - start_time
    print("\n==== RESUMEN DE ELIMINACIÓN ====")
    print(f"Total de carpetas encontradas: {stats['total_folders_found']}")
    print(f"Carpetas eliminadas con éxito: {stats['folders_deleted']}")
    print(f"Carpetas con errores: {stats['folders_failed']}")
    print(f"Tiempo total: {total_time:.2f} segundos")
    
    return stats

def main():
    """
    Función principal para eliminar carpetas Keyframe
    """
    # Configurar ruta base donde están los datos procesados
    base_path = r'C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data'
    
    print("🚀 Iniciando búsqueda de carpetas 'Keyframe' para eliminar...")
    
    # Opción para mostrar ejemplos de carpetas antes de proceder
    show_examples = input("¿Deseas ver ejemplos de las carpetas que se encontrarán? (s/n): ")
    if show_examples.lower() == 's':
        keyframe_pattern = os.path.join(base_path, "**", "Keyframe")
        keyframe_folders = glob.glob(keyframe_pattern, recursive=True)
        
        if keyframe_folders:
            print("\nEjemplos de carpetas que serán eliminadas:")
            for i, folder in enumerate(keyframe_folders[:5], 1):
                print(f"  {i}. {folder}")
            
            if len(keyframe_folders) > 5:
                print(f"  ... y {len(keyframe_folders) - 5} carpetas más.")
        else:
            print("No se encontraron carpetas 'Keyframe'.")
    
    # Preguntar si quiere continuar
    proceed = input("\n¿Deseas continuar con la eliminación? (s/n): ")
    if proceed.lower() != 's':
        print("❌ Operación cancelada.")
        return
    
    # Ejecutar la eliminación
    stats = delete_keyframe_folders(base_path)
    
    # Guardar estadísticas en archivo (opcional)
    try:
        import json
        with open(os.path.join(base_path, "eliminacion_keyframe_stats.json"), "w") as f:
            json.dump(stats, f, indent=2)
        print(f"📊 Estadísticas guardadas en {os.path.join(base_path, 'eliminacion_keyframe_stats.json')}")
    except Exception as e:
        print(f"⚠️ No se pudieron guardar las estadísticas: {str(e)}")
    
    print("\n✅ Proceso de eliminación completado.")

# Ejecutar si se llama directamente
if __name__ == "__main__":
    main()

🚀 Iniciando búsqueda de carpetas 'Keyframe' para eliminar...

Ejemplos de carpetas que serán eliminadas:
  1. C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_1\Keyframe
  2. C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_2\Keyframe
  3. C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_3\Keyframe
  4. C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_4\Keyframe
  5. C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\Aburrido\muestra_1\Keyframe
  ... y 71 carpetas más.
🔍 Se encontraron 76 carpetas 'Keyframe'.
[1/76] Eliminando: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_1\Keyframe
✅ Carpeta eliminada (1 arc